In [15]:
import numpy as np
import pandas as pd
import timeit
import time
import json
import re
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import psycopg2
import sys, os
import pandas.io.sql as psql
import sqlalchemy
from sqlalchemy import create_engine

import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import normalize

import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

from word_match import regex_matcher
from document import survey_doc
from classify import classify_labels

ModuleNotFoundError: No module named 'classify'

In [2]:
docs = classify_labels()
print(len(docs.ul_df))
print(len(docs.l_df))

/home/matt_valley/anaconda3/envs/spacy/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


83060
123


In [3]:
docs.train_test(docs, model='tfidf')

83060 123
(83060, 1000) (123, 165)


ValueError: Found input variables with inconsistent numbers of samples: [83060, 123]

In [4]:
# load all data


paths = ['~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2017 to mid 2018 comments.csv',
                 '~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2018 to mid 2019 comments.csv']

q1_df = pd.DataFrame()
q2_df = pd.DataFrame()
for data_path in paths:
    data = survey_doc(data_path)
    data.clean_unlabelled_data()
    data_q1 = data.df[(data.df['QID']==61) | (data.df['QID']=='Unique / Unusual')]
    q1_df = q1_df.append(data_q1, ignore_index=True)
    data_q2 = data.df[(data.df['QID']==62) | (data.df['QID']=='One Change')]
    q2_df = q2_df.append(data_q2, ignore_index=True)

/home/matt_valley/anaconda3/envs/spacy/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [ ]:
q1_q2_df = q1_df.append(q2_df, ignore_index=True)
print(len(q1_q2_df))

In [59]:
df = pd.read_pickle('~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/regex_scored_df.pkl')
df.head()

df = df.sort_values('comment_idx')
df = df.reset_index(drop=True)
print(df.keys())
com_ids = df.comment_idx.unique()
labels = []
idx = []
for id in com_ids:
    idx.append(id)
    com_df = df[df['comment_idx']==id]
    com_labs = com_df.topic.unique()
    labels.append(com_labs)
    
multilabel_df = pd.DataFrame()
multilabel_df['comment_idx'] = idx
multilabel_df['labels'] = labels

onehot = pd.get_dummies(multilabel_df.labels.apply(pd.Series).stack()).sum(level=0)

Index(['comment_idx', 'topic', 'conforming_text', 'matched_text',
       'context_span', 'text'],
      dtype='object')


In [60]:
print(len(onehot))
print(len(multilabel_df))

4495
4495


In [61]:
multilabel_df = pd.concat([multilabel_df, onehot], axis=1)
multilabel_df

,comment_idx,labels,Benefits and leave,Co-workers/teamwork,Commmute,Communication,Customers,Employee relations,Facility/setting,Learning & Development,Management,Pay,Purpose,Quality of care,Recognition,Schedule,Staffing level,Support and resources
0,22,[Management],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,24,[Co-workers/teamwork],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,100,"[Customers, Purpose]",0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,195,[Co-workers/teamwork],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,225,"[Co-workers/teamwork, Customers]",0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4490,184163,"[Staffing level, Benefits and leave]",1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4491,184186,[Pay],0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4492,184210,[Staffing level],0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4493,184230,[Learning & Development],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [3]:
labels = df.topic.unique()
multilabel_df.head()
for l in labels:
    l_count = 0
    for lab in multilabel_df['labels']:
        if l in lab:
            l_count+=1
    print(str(l) + ' ' + str(l_count))

Management 831
Co-workers/teamwork 1308
Customers 1157
Purpose 83
Support and resources 142
Benefits and leave 309
Facility/setting 136
Commmute 40
Quality of care 208
Staffing level 938
Employee relations 27
Communication 205
Learning & Development 190
Recognition 142
Pay 606
Schedule 100


In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [6]:
q1_q2_df.to_sql('unlabelled_data', engine)

2020-01-29 15:10:33,176 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-01-29 15:10:33,177 INFO sqlalchemy.engine.base.Engine ()
2020-01-29 15:10:33,179 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-01-29 15:10:33,180 INFO sqlalchemy.engine.base.Engine ()
2020-01-29 15:10:33,182 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("unlabelled_data")
2020-01-29 15:10:33,183 INFO sqlalchemy.engine.base.Engine ()
2020-01-29 15:10:33,185 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("unlabelled_data")
2020-01-29 15:10:33,186 INFO sqlalchemy.engine.base.Engine ()
2020-01-29 15:10:33,189 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE unlabelled_data (
	"index" BIGINT, 
	"QID" TEXT, 
	text TEXT
)


2020-01-29 15:10:33,190 INFO sqlalchemy.engine.base.Engine ()
2020-01-29 15:10:33,191 INFO sqlalchemy.engine.base.Engine COMMIT
2020-01-29 15:10:33,192 INFO sqlalchemy.engine

In [2]:
host_name = "matt-valley-insight-project-data.cubypj2k3lg1.us-west-2.rds.amazonaws.com"
user_name = "mtv2101"
password = "i0L27zWbGbhG"
database = 'matt-valley-insight-project-data'
port = 5432

engine = psycopg2.connect(
    database=database,
    user=user_name,
    password=password,
    host=host_name,
    port='5432'
)

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "matt-valley-insight-project-data.cubypj2k3lg1.us-west-2.rds.amazonaws.com" (52.11.41.24) and accepting
	TCP/IP connections on port 5432?


In [ ]:
q1_q2_df.to_sql('unlabelled_data', engine)

In [3]:
# load labelled data
data_path = ['~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/Labeled data and sentiments 2017-18 data2.csv']
labeled_data = survey_doc(data_path[0], header=0)
labeled_data.clean_labelled_data()


Index(['Comment ID', 'Customer ID', 'Survey ID', 'Respondent ID', 'QID',
       'Comment', 'JK label', 'JK sentiment', 'Unnamed: 8'],
      dtype='object')


In [5]:
labeled_data.df.head()

,QID,text,JK label,JK sentiment
110,62,Clearer communication about expectations and s...,a posted schedule online would be incredibly ...,Support and resources


In [ ]:
survey_path_1 = '/home/matt_valley/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2017 to mid 2018 comments.csv'
survey_path_2 = '/home/matt_valley/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2018 to mid 2019 comments.csv'

df = pd.read_csv(survey_path_1, header=1)
df2 = pd.read_csv(survey_path_2, header=1)

print(len(df))
df = df.dropna(how='any')
df2 = df2.dropna(how='any')
print(len(df))

df = df.rename(columns={"Comment": "text"})
df2 = df2.rename(columns={"Comment": "text"})

text_clean = [len(t) for t in df.text.values]


# load spacy model

model = 'en_core_web_sm' # for testing on laptop
#model = 'en_core_web_lg'
#model = 'en_vectors_web_lg' # many more words
nlp = spacy.load(model)
#sentencizer = nlp.create_pipe("sentencizer")
#nlp.add_pipe(sentencizer)

In [ ]:
start_time = timeit.default_timer()

match = regex_matcher()
match_df = pd.DataFrame()
for t,text in enumerate(df['text']):
    doc = nlp(text)
    out_df = match.match_topics(t, doc)
    match_df = match_df.append(out_df)
    
match_df.reset_index(drop=True, inplace=True)

process_time = timeit.default_timer() - start_time
print(str(len(df)) + ' submissions, query took ' + str(process_time) + ' seconds')

In [ ]:
print(len(df))
print(len(match_df))
pd.to_pickle(match_df, '~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/match_df.pkl')

In [6]:
match_df = pd.read_pickle('~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/match_df.pkl')

In [7]:
match_df.keys()

Index(['comment_idx', 'topic', 'conforming_text', 'matched_text',
       'context_span', 'comment_text'],
      dtype='object')

In [ ]:
### LOAD EMBEDDING MODEL

model = 'en_vectors_web_lg'
nlp_vec = spacy.load(model)

y_vec = []
topics = match_df['topic'].unique()
for topic in topics:
    y_doc = nlp(topic)
    y_vec.append(y_doc.vector)

In [ ]:
y_vec = np.array(y_vec)
y_vec.shape

In [ ]:
# BUILD CONTEXT VECTORS

X = []
for n,text in enumerate(match_df['context_span'].values):
    X.append(nlp(text).vector)
X = np.array(X)
print(X.shape)

In [ ]:
# dimensionality reduce

pca = PCA(n_components=20, whiten=True, random_state=42)
pca.fit(X, y)

In [ ]:
plt.plot(pca.explained_variance_)

In [ ]:
'''
do tfidf
'''

all_tfidf = []
topics = match_df['topic'].unique()
for topic in topics:
    temp_df = match_df[match_df['topic']==topic]
    data = [sent for sent in temp_df.comment_text]
    
    n_features = 1000
    n_components = 12
    n_top_words = 8
    
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                       max_features=n_features,
                                       stop_words='english')

    t0 = time.time()
    tfidf = tfidf_vectorizer.fit_transform(data)

    feature_array = np.array(tfidf_vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(tfidf.toarray()).flatten()[::-1]
    
    n = 10
    top_n = feature_array[tfidf_sorting][:n]
    print(topic)
    print(top_n)
        
    all_tfidf.append(tfidf)
    print("done in %0.3fs." % (time.time() - t0))


In [ ]:
from scipy.sparse import csr_matrix
X = np.empty((0,all_tfidf[0].shape[1]), dtype=all_tfidf[0].dtype)
y = np.empty((0))
for t,topic_vecs in enumerate(all_tfidf):
    print(topic_vecs.shape)
    print(X.shape)
    X = np.append(X,topic_vecs.toarray(), axis=0)
    y = np.append(y,np.ones(topic_vecs.shape[0])*t, axis=0)
print(X.shape)
print(y.shape)


In [ ]:
logit = LogisticRegression(class_weight='balanced', 
                           random_state=42, 
                           multi_class='multinomial',
                           verbose=1,
                           max_iter=1000)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X, y):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    logit.fit(X_train, y_train)
    y_pred = logit.predict((X_test))
    
    print('accuracy = ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('balenced accuracy = ' + str(metrics.balanced_accuracy_score(y_test, y_pred)))
    print('macro precision = ' + str(metrics.precision_score(y_logit.fit(X_train, y_train)

In [ ]:
y_pred = logit.predict((X_test))
print('accuracy = ' + str(metrics.accuracy_score(y_test, y_pred)))
print('balenced accuracy = ' + str(metrics.balanced_accuracy_score(y_test, y_pred)))
print('macro precision = ' + str(metrics.precision_score(y_test, y_pred,
                           average='macro')))
print('recall score = ' + str(metrics.recall_score(y_test, y_pred,
                           average='macro')))

In [ ]:
                               average='macro')))
    print('recall score = ' + str(metrics.recall_score(y_test, y_pred,
                               average='macro')))
    
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    stratify=y,
                                                    test_size=0.1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [3]:

### view hand scored data

hs = pd.read_pickle('~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/hand_scored_df.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/home/matt_valley/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/hand_scored_df.pkl'

In [13]:
print(len(hs[hs['score']=='n']))

9


ModuleNotFoundError: No module named 'example_psql'